## 1. Carregando dados 

In [1]:
# CONFIGS INICIAIS
import pandas as pd

# Caminho padrão arquivos processados
path = "../dados/processed"

# Carregar Deputados
df_deputados = pd.read_csv(f"{path}/deputados_legislatura_56.csv")
# Mantém apenas o último partido registrado por deputado
df_deputados = df_deputados.sort_values("siglaPartido").drop_duplicates("id_deputado", keep="last")

# Carregar partidos
df_partidos = pd.read_csv(f"{path}/partidos.csv")

# Carrega partidos válidos
partidos_validos = df_partidos["sigla_partido"].unique()
# Filtra deputados que estão em partidos reconhecidos
df_deputados = df_deputados[df_deputados["siglaPartido"].isin(partidos_validos)]



# Carregar proposições
df_proposicoes = pd.concat([
    pd.read_csv(f"{path}/proposicoes_{ano}.csv") for ano in range(2019,2023)
    ])

# Carregar votacoes
df_votacoes = pd.concat([
    pd.read_csv(f"{path}/votacoes_proposicoes_{ano}.csv") for ano in range(2019, 2023)
])

# Carregar votos individuais
df_votos = pd.concat([
    pd.read_csv(f"{path}/votos_individuais_proposicoes_{ano}.csv") for ano in range(2019, 2023)
])

# Carregar orientacoes
df_orientacoes = pd.concat([
    pd.read_csv(f"{path}/orientacoes_proposicoes_{ano}.csv") for ano in range(2019, 2023)
])

# Carregar autores
df_autores = pd.concat([
    pd.read_csv(f"{path}/autores_proposicoes_{ano}.csv") for ano in range(2019, 2023)
])

# Carregar temas
df_temas = pd.concat([
    pd.read_csv(f"{path}/temas_proposicoes_{ano}.csv") for ano in range(2019, 2023)
])

df = df_votos

## 2. União dos dados

### a) votos + deputados

In [2]:
# Agrupando com deputados
df = df.merge(df_deputados, on="id_deputado", how="left")

In [3]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,dataRegistroVoto,nome_deputado,siglaPartido,siglaUf,idLegislatura,email
0,946475-131,62881,1.0,2020-12-21T17:43:18,Danilo Forte,UNIÃO,CE,56.0,NaN
1,946475-131,66179,1.0,2020-12-21T17:43:18,Norma Ayub,UNIÃO,ES,56.0,NaN
2,946475-131,68720,1.0,2020-12-21T17:43:18,Fábio Henrique,UNIÃO,SE,56.0,NaN
3,946475-131,72442,Abstenção,2020-12-21T17:43:18,Felipe Carreras,PSB,PE,56.0,NaN
4,946475-131,73433,1.0,2020-12-21T17:43:18,Arlindo Chinaglia,PT,SP,56.0,NaN


In [4]:
df = df.drop(columns=["nome_deputado", "idLegislatura", "email"])
# convertendo a data apenas para dia/mês/ano
df["dataRegistroVoto"] = pd.to_datetime(df["dataRegistroVoto"]).dt.date


In [5]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,dataRegistroVoto,siglaPartido,siglaUf
0,946475-131,62881,1.0,2020-12-21,UNIÃO,CE
1,946475-131,66179,1.0,2020-12-21,UNIÃO,ES
2,946475-131,68720,1.0,2020-12-21,UNIÃO,SE
3,946475-131,72442,Abstenção,2020-12-21,PSB,PE
4,946475-131,73433,1.0,2020-12-21,PT,SP


### b) merge com partidos

In [6]:
# Após merge com deputados (que tem siglaPartido), podemos trazer o id do partido:
df = df.merge(df_partidos[["id_partido", "sigla_partido"]], 
              left_on="siglaPartido", 
              right_on="sigla_partido", 
              how="left")

# Dropa sigla_partido redundante
df = df.drop(columns=["siglaPartido"])#, "sigla_partido"

In [ ]:
#df = df.dropna(subset=["siglaUf"])

In [7]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,dataRegistroVoto,siglaUf,id_partido,sigla_partido
0,946475-131,62881,1.0,2020-12-21,CE,38009.0,UNIÃO
1,946475-131,66179,1.0,2020-12-21,ES,38009.0,UNIÃO
2,946475-131,68720,1.0,2020-12-21,SE,38009.0,UNIÃO
3,946475-131,72442,Abstenção,2020-12-21,PE,36832.0,PSB
4,946475-131,73433,1.0,2020-12-21,SP,36844.0,PT


### c) merge com detalhes votacoes

In [8]:
# Usamos df_votos (votos_individuais_proposicoes) como base para os próximos 

# Agrupando com votacoes
df = df.merge(df_votacoes, on= "id_votacao", how="left")

In [9]:
df = df.drop(columns=["dataRegistroVoto", "data_hora_registro", "descricao"], errors="ignore")

In [10]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,descricao_proposicao,aprovacao
0,946475-131,62881,1.0,CE,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0
1,946475-131,66179,1.0,ES,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0
2,946475-131,68720,1.0,SE,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0
3,946475-131,72442,Abstenção,PE,36832.0,PSB,946475,2020-12-21,PLEN,NaN,1.0
4,946475-131,73433,1.0,SP,36844.0,PT,946475,2020-12-21,PLEN,NaN,1.0


In [11]:
# printando as que tem o id_votacao unico
df[df.duplicated(subset="id_votacao", keep=False) == False].head(10)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,descricao_proposicao,aprovacao
26418,2190518-34,204365,0.0,SC,37901.0,NOVO,2190518,2021-09-23,CDC,NaN,1.0
392841,2273823-27,156190,0.0,RS,37901.0,NOVO,2273823,2021-08-11,CFFC,NaN,1.0
566140,2300676-2,204462,0.0,RJ,38009.0,UNIÃO,2300676,2021-09-30,CMULHER,REQ 63/2021 CMULHER,1.0
583328,2305340-3,204365,0.0,SC,37901.0,NOVO,2305340,2021-11-04,CDC,SOR 1/2021 CDC,1.0
583329,2305341-2,204365,0.0,SC,37901.0,NOVO,2305341,2021-11-04,CDC,SOR 2/2021 CDC,1.0
583350,2305437-2,204365,0.0,SC,37901.0,NOVO,2305437,2021-11-04,CDC,SOR 3/2021 CDC,1.0
583351,2305440-2,204365,0.0,SC,37901.0,NOVO,2305440,2021-11-04,CDC,SOR 4/2021 CDC,1.0
736407,2334096-11,204569,1.0,AM,38009.0,UNIÃO,2334096,2022-09-01,CESUBCRE,REL 2/2022 CESUBCRE,0.0
736736,2334095-11,204569,1.0,AM,38009.0,UNIÃO,2334095,2022-09-01,CESUBCRE,REL 1/2022 CESUBCRE,0.0


### d) merge com proposicoes

In [12]:
# Agrupando com proposicoes
df = df.merge(df_proposicoes, on="id_proposicao", how="left")

In [13]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,descricao_proposicao,aprovacao,sigla_tipo,cod_tipo,numero_proposicao,ano,ementa
0,946475-131,62881,1.0,CE,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0,PL,139,5028,2019,Institui a Política Nacional de Pagamento por ...
1,946475-131,66179,1.0,ES,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0,PL,139,5028,2019,Institui a Política Nacional de Pagamento por ...
2,946475-131,68720,1.0,SE,38009.0,UNIÃO,946475,2020-12-21,PLEN,NaN,1.0,PL,139,5028,2019,Institui a Política Nacional de Pagamento por ...
3,946475-131,72442,Abstenção,PE,36832.0,PSB,946475,2020-12-21,PLEN,NaN,1.0,PL,139,5028,2019,Institui a Política Nacional de Pagamento por ...
4,946475-131,73433,1.0,SP,36844.0,PT,946475,2020-12-21,PLEN,NaN,1.0,PL,139,5028,2019,Institui a Política Nacional de Pagamento por ...


In [14]:
df = df.drop(columns=["descricao_proposicao", "sigla_tipo", "ementa"], errors="ignore")

In [15]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,aprovacao,cod_tipo,numero_proposicao,ano
0,946475-131,62881,1.0,CE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019
1,946475-131,66179,1.0,ES,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019
2,946475-131,68720,1.0,SE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019
3,946475-131,72442,Abstenção,PE,36832.0,PSB,946475,2020-12-21,PLEN,1.0,139,5028,2019
4,946475-131,73433,1.0,SP,36844.0,PT,946475,2020-12-21,PLEN,1.0,139,5028,2019


### e) orientacoes partidarias

In [16]:
# Agrupa com orientações partidárias
df = df.merge(df_orientacoes[["id_votacao", "sigla_partido", "orientacao"]], on=["id_votacao", "sigla_partido"], how="left")

In [17]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,aprovacao,cod_tipo,numero_proposicao,ano,orientacao
0,946475-131,62881,1.0,CE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN
1,946475-131,66179,1.0,ES,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN
2,946475-131,68720,1.0,SE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN
3,946475-131,72442,Abstenção,PE,36832.0,PSB,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim
4,946475-131,73433,1.0,SP,36844.0,PT,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim


### f) autores

In [18]:
# Padroniza colunas para merge
df_autores = df_autores.rename(columns={"nome_autor": "nome_deputado"})

# Junta autores com deputados para obter o id
df_autores = df_autores.merge(
    df_deputados[["id_deputado", "nome_deputado"]],
    on="nome_deputado",
    how="left"
)

# Mantém apenas id_proposicao e id_autor
df_autores = df_autores[["id_proposicao", "id_deputado"]].rename(columns={"id_deputado": "id_autor"})


In [19]:
df_autores.head(5)

,id_proposicao,id_autor
0,317970,NaN
1,538196,204488.0
2,538196,160655.0
3,530357,NaN
4,501638,NaN


In [20]:
# Agrupa com autores
df = df.merge(df_autores[["id_proposicao", "id_autor"]], on="id_proposicao", how="left")

In [21]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,aprovacao,cod_tipo,numero_proposicao,ano,orientacao,id_autor
0,946475-131,62881,1.0,CE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN
1,946475-131,66179,1.0,ES,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN
2,946475-131,68720,1.0,SE,38009.0,UNIÃO,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN
3,946475-131,72442,Abstenção,PE,36832.0,PSB,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim,NaN
4,946475-131,73433,1.0,SP,36844.0,PT,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim,NaN


In [22]:
# printando as que tem o id_votacao unico
df[df.duplicated(subset="id_votacao", keep=False) == False].head(10)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,sigla_partido,id_proposicao,data,sigla_orgao,aprovacao,cod_tipo,numero_proposicao,ano,orientacao,id_autor
64439,2190518-34,204365,0.0,SC,37901.0,NOVO,2190518,2021-09-23,CDC,1.0,139,75,2019,NaN,204530.0
1687124,2273823-27,156190,0.0,RS,37901.0,NOVO,2273823,2021-08-11,CFFC,1.0,138,8,2021,NaN,NaN
3292619,2300676-2,204462,0.0,RJ,38009.0,UNIÃO,2300676,2021-09-30,CMULHER,1.0,147,63,2021,NaN,204407.0
3721917,2305340-3,204365,0.0,SC,37901.0,NOVO,2305340,2021-11-04,CDC,1.0,611,1,2021,NaN,73441.0
3721918,2305341-2,204365,0.0,SC,37901.0,NOVO,2305341,2021-11-04,CDC,1.0,611,2,2021,NaN,205550.0
3721939,2305437-2,204365,0.0,SC,37901.0,NOVO,2305437,2021-11-04,CDC,1.0,611,3,2021,NaN,73441.0
3721940,2305440-2,204365,0.0,SC,37901.0,NOVO,2305440,2021-11-04,CDC,1.0,611,4,2021,NaN,204392.0
4882279,2334096-11,204569,1.0,AM,38009.0,UNIÃO,2334096,2022-09-01,CESUBCRE,0.0,292,2,2022,Não,NaN
4882608,2334095-11,204569,1.0,AM,38009.0,UNIÃO,2334095,2022-09-01,CESUBCRE,0.0,292,1,2022,Não,NaN


### g) temas

In [23]:
# Agrupa com temas
df_temas_grouped = df_temas.groupby("id_proposicao")["tema"].apply(lambda x: list(set(x))).reset_index()
df = df.merge(df_temas_grouped, on="id_proposicao", how="left")

In [29]:
df = df.drop(columns=["sigla_partido"])

KeyError: "['sigla_partido'] not found in axis"

In [ ]:
df.head(5)

,id_votacao,id_deputado,tipoVoto,siglaUf,id_partido,id_proposicao,data,sigla_orgao,aprovacao,cod_tipo,numero_proposicao,ano,orientacao,id_autor,tema
0,946475-131,62881,1.0,CE,38009.0,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN,"[Meio Ambiente e Desenvolvimento Sustentável, ..."
1,946475-131,66179,1.0,ES,38009.0,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN,"[Meio Ambiente e Desenvolvimento Sustentável, ..."
2,946475-131,68720,1.0,SE,38009.0,946475,2020-12-21,PLEN,1.0,139,5028,2019,NaN,NaN,"[Meio Ambiente e Desenvolvimento Sustentável, ..."
3,946475-131,72442,Abstenção,PE,36832.0,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim,NaN,"[Meio Ambiente e Desenvolvimento Sustentável, ..."
4,946475-131,73433,1.0,SP,36844.0,946475,2020-12-21,PLEN,1.0,139,5028,2019,Sim,NaN,"[Meio Ambiente e Desenvolvimento Sustentável, ..."


In [31]:
df.shape

(5126323, 15)

## 3. Salvar o DF consolidado

In [32]:
# Salvar o data frame agrupado na pasta dados
df.to_csv(f"../dados/df_consolidado.csv", index=False)
